# Metadata Analysis

In [1]:
import pandas as pd
import numpy as np
import re
%load_ext autoreload
%autoreload 2
from cord.cord19 import ResearchPapers
from pathlib import Path, PurePath
from IPython.display import display

pd.options.display.max_colwidth=200
pd.options.display.max_rows = 4000

In [2]:
data_path = Path('data') / 'CORD-19-research-challenge'
metadata_path = PurePath(data_path) / 'metadata.csv'

## Load Metadata

In [3]:
research_papers = ResearchPapers.load()

Loading metadata from data\CORD-19-research-challenge
Cleaning metadata

Indexing research papers
Creating the BM25 index from the abstracts of the papers
Use index="text" if you want to index the texts of the paper instead
Finished Indexing in 36.0 seconds


In [4]:
research_papers.since_sars()


Indexing research papers


,title,abstract,journal,authors,published,when
0,SIANN: Strain Identification by Alignment to Near Neighbors,"Next-generation sequencing is increasingly being used to study samples composed of mixtures of organisms, such as in clinical applications where the presence of a pathogen at very low abundance ma...",,Samuel Minot; Stephen D Turner; Krista L Ternus; Dana R Kadavy,2014-01-10,6 years ago
1,Spatial epidemiology of networked metapopulation: An overview,"An emerging disease is one infectious epidemic caused by a newly transmissible pathogen, which has either appeared for the first time or already existed in human populations, having the capacity t...",,Lin WANG; Xiang Li,2014-06-04,6 years ago
2,Sequencing of the human IG light chain loci from a hydatidiform mole BAC library reveals locus-specific signatures of genetic diversity,"Germline variation at immunoglobulin gene (IG) loci is critical for pathogen-mediated immunity, but establishing complete reference sequences in these regions is problematic because of segmental d...",,Corey T Watson; Karyn Meltz Steinberg; Tina A Graves-Lindsay; Rene L Warren; Maika Malig; Jacqueline E Schein; Richard K Wilson; Rob Holt; Evan Eichler; Felix Breden,2014-07-03,6 years ago
3,Bayesian mixture analysis for metagenomic community profiling.,"Deep sequencing of clinical samples is now an established tool for the detection of infectious pathogens, with direct medical applications. The large amount of data generated provides an opportuni...",,Sofia Morfopoulou; Vincent Plagnol,2014-07-25,6 years ago
4,Mapping a viral phylogeny onto outbreak trees to improve host transmission inference,Developing methods to reconstruct transmission histories for viral outbreaks could provide critical information to support locating sources of disease transmission. Phylogenetic methods used to me...,,Stephen P Velsko; Jonathan E Allen,2014-11-11,5 years ago
...,...,...,...,...,...,...
45769,Personal Protective Equipment: Protecting Health Care Providers in an Ebola Outbreak,Purpose The recent Ebola epidemic that devastated West Africa has infected and killed more health care providers than any other outbreak in the history of this virus. An improved understanding of...,Clinical Therapeutics,"Fischer, William A.; Weber, David J.; Wohl, David A.",2015-11-01,4 years ago
45770,Viruses and asthma,Background Viral respiratory infection has long been known to influence the occurrence of asthma exacerbations. Over the last 20years much effort has been put into clarifying the role that viral ...,Biochimica et Biophysica Acta (BBA) - General Subjects,"Dulek, Daniel E.; Peebles, R. Stokes",2011-11-30,8 years ago
45771,Why the WHO won't use the p-word,"There are no criteria for a pandemic, but covid-19 looks like one, says Debora MacKenzie",New Scientist,"MacKenzie, Debora",2020-03-07,3 weeks ago
45772,"Communication, transparency key as Canada faces new coronavirus threat","Communication, transparency key as Canada faces new coronavirus threat",Canadian Medical Association Journal,"Glauser, Wendy",2020-02-17,1 month ago
